In [1]:
import torch

In [2]:
# set device type 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# get the data set 
'''
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt '''

'\n!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt '

In [4]:
# read the data 
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()
    

In [5]:
# print the first 1000 characters 
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# length of the data set in characters 
print(len(text))

1115394


In [7]:
# let's make a list of all the unique characters in the text 
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(' '.join(chars))
print(vocab_size)


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
65


In [8]:
# create a map from chars to intergers 
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encoder : convert string to integers 
decode = lambda l: ''.join([itos[i] for i in l]) # convert array of interges to string 

print(encode('hii htere'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 46, 58, 43, 56, 43]
hii there


In [9]:
# let's encode the entire dataset and store in torch tensor 
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
# split the dataset into train and validation sets 
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8 # this is the size(max) of charcters chunks 
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [12]:
batch_size = 4
def get_batch(split):
    data = train_data if split == 'train' else val_data
    idx = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x,y

In [13]:
# print the batched input (xb) and target(yb)
xb,yb = get_batch('train')
print(xb)
print(yb)

tensor([[56, 39, 52, 52, 53, 59, 57,  1],
        [ 2,  1, 21,  1, 56, 43, 57, 54],
        [43, 52, 43, 56, 39, 50,  6,  1],
        [56, 39, 46, 39, 51,  2,  1, 24]])
tensor([[39, 52, 52, 53, 59, 57,  1, 39],
        [ 1, 21,  1, 56, 43, 57, 54, 43],
        [52, 43, 56, 39, 50,  6,  1, 57],
        [39, 46, 39, 51,  2,  1, 24, 53]])


In [14]:
import torch 
import torch.nn as nn
from torch.nn import functional as F 
# this model only look at the last char of the token and try to predict the next char (and repeat it)
# during the forwrd pass, it compares the current char with the next char (via cross-entrophy loss)
# and in the generation phase, it predict the next char, entirely based on the last char (and repeat)
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [15]:
# instantiate the model 
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.5457, grad_fn=<NllLossBackward0>)


In [16]:
idx = torch.zeros((1,1),dtype=torch.long) # start with zero char is decent things to do(as it the new charcter "\n")
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))


pqcW$YkhyfiKa,l':auaMVYxQPCECtw;ijycfYeoJwgpflTGndkIEH$RDIGjSStEUP
jNezSQufrKkuoiKHDbR
!ik.:bYEuLvzD


In [17]:
# let's train the simple bigram model 

# torch optimizer 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) 
batch_size = 32 

for step in range(1000):
    xb,yb = get_batch('train') # get a batch of data
    
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())
    

3.6787240505218506


In [18]:
# how to implement attention, we want to find a way to communicate with all the previous chars (with the current char)
# here we simply impliment average weighting of the previous chars 

# consider the following toy example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
print(x.shape)

# average attention 
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei,dim=-1)
xbow = wei @ x

print(xbow.shape)

torch.Size([4, 8, 2])
torch.Size([4, 8, 2])


In [20]:
# self-attention  

torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# single head self-attention 
head_size = 16 
key   = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)

# key and query 
k = key(x)     # (B,T,16)
q = query(x)   # (B,T,16)

# weights, we need to divide the weights by sqrt (head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5 # (B,T,16) @ (B, 16,T) -> (B,T,T)
tril = torch.tril(torch.ones(T,T))
wei  = wei.masked_fill(tril == 0, float('-inf'))
wei  = F.softmax(wei,dim=-1)

# value 
v = value(x)

# output 
out = wei @ v

print(out.shape)


torch.Size([4, 8, 16])


In [21]:
# why we need to divide the weights by sqrt(head_size)? 

# when the values are small 
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1))

# when the valus are large, then the output gets too peaky, converging to one hot 
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])


tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [26]:
x = torch.randn(32, 100)
x.mean(1,keepdim=True).shape

torch.Size([32, 1])

### Batch-Normalization: How it works ? 

1. For each mini-batch of data:

* Calculate the mean and variance of the activations for each feature.
* Normalize the activations by subtracting the mini-batch mean and dividing by the mini-batch standard deviation.
* Scale and shift the normalized values using learnable parameters (gamma and beta) unique to each layer.

In [29]:
# the input is (B,C), and output is also (B,C)
# in pytorch implimentation of "batchnorm1d" it can handle shapes of (B,C,L), where L is the sequance length 

class LayerNorm1d: # (used to be BatchNorm1d)

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True)   # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])